```@meta
CurrentModule = CounterfactualExplanations 
```


In [ ]:
#| echo: false
#| output: false
include("$(pwd())/docs/setup_docs.jl")
eval(setup_docs)

# Performance Evaluation

Now that we know how to generate counterfactual explanations in Julia, you may have a few follow-up questions: How do I know if the counterfactual search has been successful? How good is my counterfactual explanation? What does 'good' even mean in this context? In this tutorial, we will see how counterfactual explanations can be evaluated with respect to their performance.

## Default Measures

Numerous evaluation measures for counterfactual explanations have been proposed. In what follows, we will cover some of the most important measures.

### Single Measure, Single Counterfactual

One of the most important measures is [`validity`](@ref), which simply determines whether or not a counterfactual explanation $x^{\prime}$ is valid in the sense that it yields the target prediction: $M(x^{\prime})=t$. We can evaluate the validity of a single counterfactual explanation `ce` using the [`Evaluation.evaluate`](@ref) function as follows:


In [ ]:
using CounterfactualExplanations.Evaluation: evaluate, validity
evaluate(ce; measure=validity)

For a single counterfactual explanation, this evaluation measure can only take two values: it is either equal to `1`, if the explanation is valid or `0` otherwise. Another important measure is [`distance`](@ref), which relates to the distance between the factual $x$ and the counterfactual $x^{\prime}$. In the context of Algorithmic Recourse, higher distances are typically associated with higher costs to individuals seeking recourse. 


In [ ]:
using CounterfactualExplanations.Evaluation: distance
evaluate(ce; measure=distance)

By default, `distance` computes the L2 (Euclidean) distance. 

### Multiple Measures, Single Counterfactual

You might be interested in computing not just the L2 distance, but various LP norms. This can be done by supplying a vector of functions to the `measure` key argument. For convenience, all default distance measures have already been collected in a vector:


In [ ]:
using CounterfactualExplanations.Evaluation: distance_measures
distance_measures

We can use this vector of evaluation measures as follows:


In [ ]:
evaluate(ce; measure=distance_measures)

If no `measure` is specified, the `evaluate` method will return all default measures,


In [ ]:
evaluate(ce)

which include:


In [ ]:
CounterfactualExplanations.Evaluation.default_measures

### Multiple Measures and Counterfactuals

We can also evaluate multiple counterfactual explanations at once:


In [ ]:
generator = DiCEGenerator()
ces = generate_counterfactual(x, target, counterfactual_data, M, generator; num_counterfactuals=5)
evaluate(ces)

By default, each evaluation measure is aggregated across all counterfactual explanations. To return individual measures for each counterfactual explanation you can specify `report_each=true`


In [ ]:
evaluate(ces; report_each=true)

## Custom Measures

A `measure` is just a method that takes a `CounterfactualExplanation` as its only positional argument and `agg::Function` as a key argument specifying how measures should be aggregated across counterfactuals. Defining custom measures is therefore straightforward. For example, we could define a measure to compute the inverse target probability as follows:


In [ ]:
my_measure(ce::CounterfactualExplanation; agg=mean) = agg(1 .- CounterfactualExplanations.target_probs(ce))
evaluate(ce; measure=my_measure)

## Tidy Output

By default, `evaluate` returns vectors of evaluation measures. The optional key argument `output_format::Symbol` can be used to post-process the output in two ways: firstly, to return the output as a dictionary, specify `output_format=:Dict`:


In [ ]:
evaluate(ces; output_format=:Dict, report_each=true)

Secondly, to return the output as a data frame, specify `output_format=:DataFrame`.


In [ ]:
evaluate(ces; output_format=:DataFrame, report_each=true)

By default, data frames are pivoted to long format using individual counterfactuals as the `id` column. This behaviour can be suppressed by specifying `pivot_longer=false`.

## Multiple Counterfactual Explanations

It may be necessary to generate counterfactual explanations for multiple individuals. 


In [ ]:
#| echo: false
n_individuals = 5
Markdown.parse(
    """
    Below, for example, we first select multiple samples ($n_individuals) from the non-target class and then generate counterfactual explanations for all of them. 
    """
)

In [ ]:
# Factual and target:
ids = rand(findall(predict_label(M, counterfactual_data) .== factual), n_individuals)
xs = select_factual(counterfactual_data, ids)
ces = generate_counterfactual(xs, target, counterfactual_data, M, generator; num_counterfactuals=5)
evaluation = evaluate(ces)

In [ ]:
#| echo: false

show(evaluation)

This leads us to our next topic: Performance Benchmarks.